# 📊 Liquidation Evolution Over Time - Visual Analysis
## Interactive Time-Series Charts: Day, Time, Volume Evolution

This notebook provides **visual time-series analysis** of liquidations showing:
- **Daily patterns** - How liquidations change day-by-day
- **Hourly evolution** - Intraday liquidation patterns
- **Exchange comparison** - Binance vs Bybit over time
- **Value evolution** - USD volume trends
- **Cascade patterns** - When and how cascades occur

In [ ]:
# Import dependencies
import asyncio
import asyncpg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import seaborn as sns

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Dependencies imported")

In [ ]:
# Connect to database
async def get_db_connection():
    return await asyncpg.connect(
        host='localhost',
        port=5432,
        database='liquidations',
        user='screener-m3'
    )

conn = await get_db_connection()
print("✅ Database connected")

## 📈 Chart 1: Liquidation Volume Evolution Over Time

In [ ]:
# Get hourly data for last 7 days
query = """
SELECT
    hour,
    exchange,
    symbol,
    side,
    event_count,
    total_value_usd,
    total_quantity
FROM liquidations_hourly
WHERE hour >= NOW() - INTERVAL '7 days'
ORDER BY hour ASC
"""

rows = await conn.fetch(query)
df_hourly = pd.DataFrame([dict(row) for row in rows])

print(f"Loaded {len(df_hourly)} hourly data points")
df_hourly.head()

In [ ]:
# Interactive time-series chart: Volume over time by exchange
if not df_hourly.empty:
    fig = go.Figure()

    for exchange in df_hourly['exchange'].unique():
        exchange_data = df_hourly[df_hourly['exchange'] == exchange]
        hourly_totals = exchange_data.groupby('hour')['total_value_usd'].sum().reset_index()

        color = 'gold' if exchange == 'binance' else 'cyan'

        fig.add_trace(go.Scatter(
            x=hourly_totals['hour'],
            y=hourly_totals['total_value_usd'],
            mode='lines+markers',
            name=exchange.upper(),
            line=dict(color=color, width=3),
            marker=dict(size=6),
            hovertemplate=
                '<b>%{fullData.name}</b><br>' +
                'Time: %{x}<br>' +
                'Volume: $%{y:,.0f}<br>' +
                '<extra></extra>'
        ))

    fig.update_layout(
        title=dict(
            text='💰 Liquidation Volume Evolution - By Exchange (Last 7 Days)',
            font=dict(size=20)
        ),
        xaxis_title='Date & Time',
        yaxis_title='Total USD Value',
        hovermode='x unified',
        height=600,
        template='plotly_dark',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    fig.show()
else:
    print("⏳ No data yet. Start the aggregator and wait for liquidations.")

## 📊 Chart 2: Daily Liquidation Patterns (Heatmap)

In [ ]:
# Create daily heatmap showing liquidation activity by hour of day
if not df_hourly.empty:
    # Extract day of week and hour
    df_hourly['day_of_week'] = pd.to_datetime(df_hourly['hour']).dt.day_name()
    df_hourly['hour_of_day'] = pd.to_datetime(df_hourly['hour']).dt.hour

    # Aggregate by day of week and hour
    heatmap_data = df_hourly.groupby(['day_of_week', 'hour_of_day'])['total_value_usd'].sum().reset_index()

    # Pivot for heatmap
    heatmap_pivot = heatmap_data.pivot(index='day_of_week', columns='hour_of_day', values='total_value_usd')

    # Order days correctly
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    heatmap_pivot = heatmap_pivot.reindex([d for d in day_order if d in heatmap_pivot.index])

    # Create plotly heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='YlOrRd',
        hovertemplate='Day: %{y}<br>Hour: %{x}:00<br>Volume: $%{z:,.0f}<extra></extra>'
    ))

    fig.update_layout(
        title='🔥 Liquidation Heatmap - Day of Week vs Hour of Day',
        xaxis_title='Hour of Day (UTC)',
        yaxis_title='Day of Week',
        height=500,
        template='plotly_dark'
    )

    fig.show()

## ⚡ Chart 3: Cascade Events Over Time

In [ ]:
# Get cascade data
query_cascades = """
SELECT
    cascade_start as time,
    cascade_id::text,
    symbol,
    event_count,
    total_value_usd,
    max_risk_score,
    exchange_count,
    exchanges_involved,
    duration_seconds
FROM liquidation_cascades
WHERE hour >= NOW() - INTERVAL '7 days'
ORDER BY cascade_start ASC
"""

cascade_rows = await conn.fetch(query_cascades)
df_cascades = pd.DataFrame([dict(row) for row in cascade_rows])

if not df_cascades.empty:
    print(f"Found {len(df_cascades)} cascades in last 7 days")

    # Create scatter plot with size = value, color = risk score
    df_cascades['is_cross_exchange'] = df_cascades['exchange_count'] > 1

    fig = px.scatter(
        df_cascades,
        x='time',
        y='total_value_usd',
        size='event_count',
        color='max_risk_score',
        color_continuous_scale='Reds',
        hover_data=['event_count', 'duration_seconds', 'exchange_count'],
        title='🚨 Cascade Events Over Time (Size = Event Count, Color = Risk Score)',
        labels={
            'time': 'Date & Time',
            'total_value_usd': 'Total USD Value',
            'max_risk_score': 'Max Risk Score'
        },
        template='plotly_dark',
        height=600
    )

    fig.update_traces(
        marker=dict(
            line=dict(width=2, color='DarkSlateGrey'),
            sizemode='diameter',
            sizeref=0.5
        )
    )

    fig.show()
else:
    print("⏳ No cascade data yet")

## 📉 Chart 4: Long vs Short Liquidations Over Time

In [ ]:
# Compare LONG vs SHORT liquidations over time
if not df_hourly.empty:
    # Aggregate by hour and side
    side_data = df_hourly.groupby(['hour', 'side'])['total_value_usd'].sum().reset_index()

    fig = go.Figure()

    for side in ['LONG', 'SHORT']:
        side_subset = side_data[side_data['side'] == side]
        color = 'red' if side == 'LONG' else 'green'

        fig.add_trace(go.Scatter(
            x=side_subset['hour'],
            y=side_subset['total_value_usd'],
            mode='lines',
            name=f'{side} Liquidations',
            line=dict(color=color, width=3),
            fill='tozeroy',
            hovertemplate='%{fullData.name}<br>Time: %{x}<br>Volume: $%{y:,.0f}<extra></extra>'
        ))

    fig.update_layout(
        title='📊 LONG vs SHORT Liquidations Over Time',
        xaxis_title='Date & Time',
        yaxis_title='Total USD Value',
        hovermode='x unified',
        height=600,
        template='plotly_dark'
    )

    fig.show()

## 🏦 Chart 5: Exchange Market Share Evolution

In [ ]:
# Show how market share changes over time
if not df_hourly.empty:
    # Calculate total by hour
    hourly_totals = df_hourly.groupby(['hour', 'exchange'])['total_value_usd'].sum().reset_index()

    # Calculate percentage
    total_by_hour = hourly_totals.groupby('hour')['total_value_usd'].sum()
    hourly_totals['percentage'] = hourly_totals.apply(
        lambda row: (row['total_value_usd'] / total_by_hour[row['hour']] * 100)
        if total_by_hour[row['hour']] > 0 else 0,
        axis=1
    )

    fig = px.area(
        hourly_totals,
        x='hour',
        y='percentage',
        color='exchange',
        title='🏦 Exchange Market Share Evolution (% of Total Volume)',
        labels={'hour': 'Date & Time', 'percentage': 'Market Share (%)'},
        color_discrete_map={'binance': 'gold', 'bybit': 'cyan'},
        template='plotly_dark',
        height=600
    )

    fig.update_traces(
        hovertemplate='%{fullData.name}<br>Time: %{x}<br>Share: %{y:.1f}%<extra></extra>'
    )

    fig.show()

## 📅 Chart 6: Daily Summary - Multi-Panel View

In [ ]:
# Create comprehensive daily view
if not df_hourly.empty:
    # Aggregate by day
    df_hourly['date'] = pd.to_datetime(df_hourly['hour']).dt.date
    daily_data = df_hourly.groupby(['date', 'exchange']).agg({
        'total_value_usd': 'sum',
        'total_quantity': 'sum',
        'event_count': 'sum'
    }).reset_index()

    # Create subplots
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            '💰 Daily USD Volume',
            '🪙 Daily BTC Volume',
            '📊 Daily Event Count'
        ),
        vertical_spacing=0.1
    )

    for exchange in daily_data['exchange'].unique():
        exchange_data = daily_data[daily_data['exchange'] == exchange]
        color = 'gold' if exchange == 'binance' else 'cyan'

        # USD Volume
        fig.add_trace(
            go.Bar(
                x=exchange_data['date'],
                y=exchange_data['total_value_usd'],
                name=exchange.upper(),
                marker_color=color,
                legendgroup=exchange,
                hovertemplate='%{x}<br>$%{y:,.0f}<extra></extra>'
            ),
            row=1, col=1
        )

        # BTC Volume
        fig.add_trace(
            go.Bar(
                x=exchange_data['date'],
                y=exchange_data['total_quantity'],
                name=exchange.upper(),
                marker_color=color,
                legendgroup=exchange,
                showlegend=False,
                hovertemplate='%{x}<br>%{y:.4f} BTC<extra></extra>'
            ),
            row=2, col=1
        )

        # Event Count
        fig.add_trace(
            go.Bar(
                x=exchange_data['date'],
                y=exchange_data['event_count'],
                name=exchange.upper(),
                marker_color=color,
                legendgroup=exchange,
                showlegend=False,
                hovertemplate='%{x}<br>%{y} events<extra></extra>'
            ),
            row=3, col=1
        )

    fig.update_layout(
        title_text='📅 Daily Liquidation Summary - All Metrics',
        height=1000,
        template='plotly_dark',
        barmode='group'
    )

    fig.update_yaxes(title_text='USD Value', row=1, col=1)
    fig.update_yaxes(title_text='BTC Volume', row=2, col=1)
    fig.update_yaxes(title_text='Event Count', row=3, col=1)

    fig.show()

## 📈 Chart 7: Cumulative Volume Over Time

In [ ]:
# Show cumulative liquidation volume
if not df_hourly.empty:
    # Calculate cumulative sum by exchange
    fig = go.Figure()

    for exchange in df_hourly['exchange'].unique():
        exchange_data = df_hourly[df_hourly['exchange'] == exchange].copy()
        hourly_totals = exchange_data.groupby('hour')['total_value_usd'].sum().sort_index()
        cumulative = hourly_totals.cumsum()

        color = 'gold' if exchange == 'binance' else 'cyan'

        fig.add_trace(go.Scatter(
            x=cumulative.index,
            y=cumulative.values,
            mode='lines',
            name=exchange.upper(),
            line=dict(color=color, width=4),
            fill='tonexty' if exchange != 'binance' else None,
            hovertemplate='%{fullData.name}<br>Time: %{x}<br>Cumulative: $%{y:,.0f}<extra></extra>'
        ))

    fig.update_layout(
        title='📈 Cumulative Liquidation Volume Over Time',
        xaxis_title='Date & Time',
        yaxis_title='Cumulative USD Value',
        hovermode='x unified',
        height=600,
        template='plotly_dark'
    )

    fig.show()

## 🔄 Auto-Refresh

To see the latest data, re-run all cells above. The charts will update with new liquidation data.

**Tip:** Run the cells sequentially to see how liquidations evolve over time!

In [ ]:
# Close database connection
await conn.close()
print("✅ Database connection closed")